In [ ]:
import subprocess as sp
import getpass
import questionary
import random
import pymysql

In [ ]:
username = "root"
password = "Patanjali@2003"
con = pymysql.connect(host='localhost',
                              port=3306,
                              user=username,
                              password=password,
                              db='CR',
                              cursorclass=pymysql.cursors.DictCursor)

In [ ]:
if(con.open):
            print("Connected")
else:
    print("Failed to connect")

In [ ]:
cursor = con.cursor()

In [ ]:
# function to add new players
def add_players(user_name, email, trophies, level, clan_id, highest_trophies):
    try:
        date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Get the maximum PlayerID from the Player table
        cursor.execute("SELECT MAX(PlayerID) FROM Player;")
        result = cursor.fetchone()
        key = list(result.keys())[0]
        max_player_id = result[key]
        # Increment the maximum PlayerID to generate a new ID
        new_player_id = max_player_id + 1

        # SQL query to insert data into the Player table
        query = """
            INSERT INTO Player (PlayerID, User_Name, Date_Of_Join, Email, Current_Trophies, Level, ClanID, Highest_Trophies)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """

        # Tuple with the values for the query
        values = (new_player_id, user_name, date, email, trophies, level, clan_id, highest_trophies)

        # Execute the query and commit the changes
        cursor.execute(query, values)
        con.commit()
        print("Player added successfully")

    except pymysql.Error as err:
        print(f"Error: {err}")
    return


In [ ]:
def add_cards(card_id, hp, damage, elixit, level, rarity):
    # check if card_id 
    try:
        cursor.execute("SELECT * FROM Cards WHERE CardID = %s", (card_id))
        result = cursor.fetchone()
        if result is None:
            query = """
            INSERT INTO Cards (CardID, Card_Base_Hitpoints, Card_Base_Damage, Elixir_Cost, Card_Level, Card_Rarity)
            VALUES (%s, %s, %s, %s, %s, %s)
        """
            values = (card_id, hp, damage, elixit, level, rarity)
            cursor.execute(query, values)
            con.commit()
            print("Card added successfully")
        else:
            print("Card already exists")
    except pymysql.Error as err:
        print(f"Error: {err}")
    return
def delete_card(card_id):
    try:
        # SQL query to delete a player from the Player table
        query = """
            DELETE FROM Cards
            WHERE CardID = %s
        """

        # Tuple with the value for the query
        values = (card_id)

        # Execute the query and commit the changes
        cursor.execute(query, values)
        con.commit()
        print("Card deleted successfully")

    except pymysql.Error as err:
        print(f"Error: {err}")
    return


In [ ]:
def add_clan(ClanID,Clan_Name,Clan_Trophies,Description,Clan_Leader):
    try:
        # check if the Clan_Leader is in the Player table, if not, show error
        cursor.execute("SELECT * FROM Player WHERE User_Name = %s", (Clan_Leader))
        result = cursor.fetchone()
        if result is None:
            print("Clan Leader not found")
            return
        cursor.execute("SELECT * FROM Clan WHERE ClanID = %s", (ClanID))
        result = cursor.fetchone()
        if result is None:
            query = """
            INSERT INTO Clan (ClanID, Clan_Name, Clan_Trophies, Description, Clan_Leader)
            VALUES (%s, %s, %s, %s, %s)
        """
            values = (ClanID,Clan_Name,Clan_Trophies,Description,Clan_Leader)
            cursor.execute(query, values)
            con.commit()
            print("Clan added successfully")
        else:
            print("Clan already exists")
    except pymysql.Error as err:
        print(f"Error: {err}")
    return

In [ ]:
# add deck
def add_deck(player_id, card1, card2, card3, card4, deck_name):
    # forst check if player and all the cards exist
    try:
        cursor.execute("SELECT * FROM Player WHERE PlayerID = %s", (player_id))
        result = cursor.fetchone()
        if result is None:
            print("Player not found")
            return
        cursor.execute("SELECT * FROM Cards WHERE CardID = %s", (card1))
        result = cursor.fetchone()
        if result is None:
            print("Card1 not found")
            return
        cursor.execute("SELECT * FROM Cards WHERE CardID = %s", (card2))
        result = cursor.fetchone()
        if result is None:
            print("Card2 not found")
            return
        cursor.execute("SELECT * FROM Cards WHERE CardID = %s", (card3))
        result = cursor.fetchone()
        if result is None:
            print("Card3 not found")
            return
        cursor.execute("SELECT * FROM Cards WHERE CardID = %s", (card4))
        result = cursor.fetchone()
        if result is None:
            print("Card4 not found")
            return
        # get max deck id
        cursor.execute("SELECT MAX(DeckID) FROM Deck;")
        result = cursor.fetchone()
        key = list(result.keys())[0]
        max_deck_id = result[key]
        # Increment the maximum PlayerID to generate a new ID
        new_deck_id = max_deck_id + 1
        # inset into deck, DeckID, PlayerID, Deck_name;
        query = """
            INSERT INTO Deck (DeckID, PlayerID, Deck_name)
            VALUES (%s, %s, %s)
        """
        values = (new_deck_id, player_id, deck_name)
        cursor.execute(query, values)
        con.commit()
        # now add deck_id, Card1_ID, Card2_ID, Card3_ID, Card4_ID into Deck_Cards
        query = """
            INSERT INTO Deck_Cards (DeckID, Card1_ID, Card2_ID, Card3_ID, Card4_ID)
            VALUES (%s, %s, %s, %s, %s)
        """
        values = (new_deck_id, card1, card2, card3, card4)
        cursor.execute(query, values)
        con.commit()
        print("Deck added successfully")
        
    except pymysql.Error as err:
        print(f"Error: {err}")

In [ ]:
def del_clan(ClanID):
    try:
        cursor.execute("SELECT * FROM Clan WHERE ClanID = %s", (ClanID))
        result = cursor.fetchone()
        if result is None:
            print("Clan not found")
            return
        query = """
            DELETE FROM Clan WHERE ClanID = %s
        """
        cursor.execute(query, (ClanID))
        con.commit()
        print("Clan deleted successfully")
    except pymysql.Error as err:
        print(f"Error: {err}")
    return

In [ ]:
def inc_player_trophies(WinnerID):
    # pick a random number between 26 and 30
    increase = random.randint(26, 30)
    try:
        cursor.execute("SELECT * FROM Player WHERE PlayerID = %s", (WinnerID))
        result = cursor.fetchone()
        if result is None:
            print("Player not found")
            return
        query = """
            UPDATE Player SET Current_Trophies = Current_Trophies + %s WHERE PlayerID = %s
        """
        cursor.execute(query, (increase,WinnerID))
        con.commit()
        print("Player trophies increased successfully")
    except pymysql.Error as err:
        print(f"Error: {err}")
    return

In [ ]:
import random

def dec_player_trophies(LoserID):
    # pick a random number between 26 and 30
    decrease = random.randint(12, 18)
    try:
        cursor.execute("SELECT * FROM Player WHERE PlayerID = %s", (LoserID,))
        result = cursor.fetchone()
        if result is None:
            print("Player not found")
            return
        # if the player trophies are less than 12, set it to 0
        if result["Current_Trophies"] < 12:
            query = """
                UPDATE Player SET Current_Trophies = 0 WHERE PlayerID = %s
            """
            cursor.execute(query, (LoserID,))
            con.commit()
            print("Player trophies decreased successfully")
            return
        query = """
            UPDATE Player SET Current_Trophies = Current_Trophies - %s WHERE PlayerID = %s
        """
        cursor.execute(query, (decrease, LoserID))
        con.commit()
        print("Player trophies decreased successfully")
    except pymysql.Error as err:
        print(f"Error: {err}")
    return


In [ ]:
def add_battle(BattleID , Start_Time, End_Time, Arena, WinnerID , PlayerID):
    try:
        cursor.execute("SELECT * FROM Player WHERE PlayerID = %s", (PlayerID))
        result = cursor.fetchone()
        if result is None:
            print("Player not found")
            return
        cursor.execute("SELECT * FROM Player WHERE PlayerID = %s", (WinnerID))
        result = cursor.fetchone()
        if result is None:
            print("Winner not found")
            return
        cursor.execute("SELECT * FROM Arena WHERE Arena_Name = %s", (Arena))
        result = cursor.fetchone()
        if result is None:
            print("Arena not found")
            return
        cursor.execute("SELECT * FROM Battle WHERE BattleID = %s", (BattleID))
        result = cursor.fetchone()
        if result is None:
            query = """
            INSERT INTO Battle (BattleID , Start_Time, End_Time, Arena, WinnerID , PlayerID)
            VALUES (%s, %s, %s, %s, %s, %s)
        """
            values = (BattleID , Start_Time, End_Time, Arena, WinnerID , PlayerID)
            cursor.execute(query, values)
            con.commit()
            dec_player_trophies(PlayerID)
            inc_player_trophies(WinnerID)
            print("Battle added successfully")
        else:
            print("Battle already exists")
    except pymysql.Error as err:
        print(f"Error: {err}")
    
    
    return

In [ ]:
def del_player(PlayerID):
    try:
        # check if the PlayerID is in the Player table, if not, show error
        cursor.execute("SELECT * FROM Player WHERE PlayerID = %s", (PlayerID,))
        result = cursor.fetchone()
        if result is None:
            print("Player not found")
            return

        cursor.execute("SELECT User_Name FROM Player WHERE PlayerID = %s", (PlayerID,))
        player_name = cursor.fetchone()
        player_name = player_name["User_Name"]
        cursor.execute("SELECT * FROM Clan WHERE Clan_Leader = %s", (player_name,))
        result = cursor.fetchone()
        if result is None:
            print("Clan not found")
        else:
            query = """
                UPDATE Clan SET Clan_Leader = NULL WHERE Clan_Leader = %s
            """
            cursor.execute(query, (player_name,))
            con.commit()
            print("Clan_Leader updated successfully")

        cursor.execute("SELECT * FROM Player WHERE PlayerID = %s", (PlayerID,))
        result = cursor.fetchone()
        if result is None:
            print("Player not found")
            return
        query = """
            DELETE FROM Player WHERE PlayerID = %s
        """
        cursor.execute(query, (PlayerID,))
        con.commit()
        print("Player deleted successfully")
    except pymysql.Error as err:
        print(f"Error: {err}")
    return


In [ ]:
def del_battle(BattleID):
    try:
        cursor.execute("SELECT * FROM Battle WHERE BattleID = %s", (BattleID))
        result = cursor.fetchone()
        if result is None:
            print("Battle not found")
            return
        query = """
            DELETE FROM Battle WHERE BattleID = %s
        """
        cursor.execute(query, (BattleID))
        con.commit()
        print("Battle deleted successfully")
    except pymysql.Error as err:
        print(f"Error: {err}")
    return

In [ ]:
def update_card(CardID, new_level):
    try:
        # if new_level - current_level > 1, show error
        if new_level > 14:
            print("Card level cannot be greater than 14, setting it to 14")
            new_level = 14

        cursor.execute("SELECT * FROM Cards WHERE CardID = %s", (CardID))
        result = cursor.fetchone()
        if result is None:
            print("Card not found")
            return
        current_level = result["Card_Level"]
        if new_level - current_level > 1:
            print("Card level cannot be increased by more than 1")
            return
        
        if new_level - current_level <= 0:
            print("Card level cannot be decreased")
            return
        
        if new_level < 1:
            print("Card level cannot be less than 1, setting it to 1")
            new_level = 1


        
        cursor.execute("SELECT * FROM Cards WHERE CardID = %s", (CardID))
        result = cursor.fetchone()
        if result is None:
            print("Card not found")
            return
        query = """
            UPDATE Cards SET Card_Level = %s WHERE CardID = %s
        """
        cursor.execute(query, (new_level,CardID))
        query = """
            UPDATE Cards SET Card_Base_Hitpoints = Card_Base_Hitpoints * 1.1 WHERE CardID = %s
        """
        cursor.execute(query, (CardID))
        query = """
            UPDATE Cards SET Card_Base_Damage = Card_Base_Damage * 1.1 WHERE CardID = %s
        """
        cursor.execute(query, (CardID))
        con.commit()
        print("Card updated successfully")
    except pymysql.Error as err:
        print(f"Error: {err}")
    return



In [ ]:
def update_email(PlayerID, new_email):
    try:
        cursor.execute("SELECT * FROM Player WHERE PlayerID = %s", (PlayerID))
        result = cursor.fetchone()
        if result is None:
            print("Player not found")
            return
        query = """
            UPDATE Player SET Email = %s WHERE PlayerID = %s
        """
        cursor.execute(query, (new_email,PlayerID))
        con.commit()
        print("Email updated successfully")
    except pymysql.Error as err:
        print(f"Error: {err}")
    return

In [ ]:
def update_clan_leader(ClanID, PlayerID):
    try:
        cursor.execute("SELECT * FROM Player WHERE PlayerID = %s", (PlayerID))
        result = cursor.fetchone()
        if result is None:
            print("Player not found")
            return
        # get the player name
        cursor.execute("SELECT User_Name FROM Player WHERE PlayerID = %s", (PlayerID,))
        player_name = cursor.fetchone()
        player_name = player_name["User_Name"]

        # check if the player is in the given clanID, if not show error
        cursor.execute("SELECT ClanID FROM Player WHERE PlayerID = %s", (PlayerID,))
        player_clan_id = cursor.fetchone()
        player_clan_id = player_clan_id["ClanID"]
        if player_clan_id != ClanID:
            print("Player is not in the given clan, You cannot make him the leader")
            return

        cursor.execute("SELECT * FROM Clan WHERE ClanID = %s", (ClanID))
        result = cursor.fetchone()
        if result is None:
            print("Clan not found")
            return
        query = """
            UPDATE Clan SET Clan_Leader = %s WHERE ClanID = %s
        """
        cursor.execute(query, (player_name,ClanID))
        con.commit()
        print("Clan Leader updated successfully")
    except pymysql.Error as err:
        print(f"Error: {err}")
    return

In [ ]:
def change_clan(PlayerID, ClanID):
    try:
        cursor.execute("SELECT * FROM Player WHERE PlayerID = %s", (PlayerID,))
        result = cursor.fetchone()
        if result is None:
            print("Player not found")
            return
        cursor.execute("SELECT * FROM Clan WHERE ClanID = %s", (ClanID))
        result = cursor.fetchone()
        if result is None:
            print("Clan not found")
            return
        query = """
            UPDATE Player SET ClanID = %s WHERE PlayerID = %s
        """
        cursor.execute(query, (ClanID,PlayerID))
        con.commit()
        print("Clan updated successfully")
    except pymysql.Error as err:
        print(f"Error: {err}")
    return

In [ ]:
change_clan(1, 4)

In [ ]:
update_email(1, "bolt@example.com")

In [ ]:
update_clan_leader(4,1)

In [ ]:
add_players("Patanjali", "dummy@example.com", 100, 1, 1, 100)

In [ ]:
del_player(4)

In [ ]:
add_clan(4, "Clan4", 100, "Clan4", "Patanjali")

In [ ]:
add_cards(4, 2000, 550, 4, 2, "legendary")

In [ ]:
add_clan(5, "knights_2", 2000, "best clan", "k")

In [ ]:
add_cards(1, 2000, 550, 4, 2, "legendary")

In [ ]:
add_battle(7, "2021-04-20 12:00:00", "2021-04-20 12:00:00", "Gold Arena", 2, 1)

In [ ]:
update_card(1, 6)

In [ ]:
del_battle(4)
del_battle(5)
del_battle(6)


In [ ]:
# retrieval operations
# operation 1: selecttion

def select_player(player_name):
    try:
        cursor.execute("SELECT * FROM Player WHERE User_Name = %s", (player_name))
        result = cursor.fetchone()
        if result is None:
            print("Player not found")
            return
        print(result)
    except pymysql.Error as err:
        print(f"Error: {err}")
    return
def select_card(card_id):
    try:
        cursor.execute("SELECT * FROM Cards WHERE CardID = %s", (card_id))
        result = cursor.fetchone()
        if result is None:
            print("Card not found")
            return
        print(result)
    except pymysql.Error as err:
        print(f"Error: {err}")
    return
def select_clan(clan_name):
    try:
        cursor.execute("SELECT * FROM Clan WHERE Clan_Name = %s", (clan_name))
        result = cursor.fetchone()
        if result is None:
            print("Clan not found")
            return
        print(result)
    except pymysql.Error as err:
        print(f"Error: {err}")
    return
def select_deck(deck_id):
    try:
        cursor.execute("SELECT * FROM Deck WHERE DeckID = %s", (deck_id))
        result = cursor.fetchone()
        if result is None:
            print("Deck not found")
            return
        print(result)
    except pymysql.Error as err:
        print(f"Error: {err}")
    return

In [ ]:
# retrieval operations
# projection operations
# select all players above trophies x in clan clan_name
def select_player_above_trophies(trophies, clan_name):
    try:
        cursor.execute("SELECT * FROM Player WHERE Current_Trophies > %s AND ClanID = (SELECT ClanID FROM Clan WHERE Clan_Name = %s)", (trophies, clan_name))
        result = cursor.fetchall()
        if result is None:
            print("Player not found")
            return
        print(result)
    except pymysql.Error as err:
        print(f"Error: {err}")
    return

# select all cards above certain hp in a deck_id
def select_cards_deck(hp, deck_id):
    try:
        cursor.execute("SELECT * FROM Cards WHERE Card_Base_Hitpoints > %s AND CardID IN (SELECT Card1_ID FROM Deck_Cards WHERE DeckID = %s)", (hp, deck_id))
        result = cursor.fetchall()
        if result is None:
            print("Card not found")
            return
        print(result)
    except pymysql.Error as err:
        print(f"Error: {err}")
    return


In [ ]:
# aggregate functions
# count the number of players in a clan
def count_players(clan_name):
    try:
        cursor.execute("SELECT COUNT(*) FROM Player WHERE ClanID = (SELECT ClanID FROM Clan WHERE Clan_Name = %s)", (clan_name))
        result = cursor.fetchone()
        if result is None:
            print("Clan not found")
            return
        print(result)
    except pymysql.Error as err:
        print(f"Error: {err}")
    return
# avg trophies of players in a clan
def avg_trophies_clan(clan_name):
    try:
        cursor.execute("SELECT AVG(Current_Trophies) FROM Player WHERE ClanID = (SELECT ClanID FROM Clan WHERE Clan_Name = %s)", (clan_name))
        result = cursor.fetchone()
        if result is None:
            print("Clan not found")
            return
        print(result)
    except pymysql.Error as err:
        print(f"Error: {err}")
    return

In [ ]:
# function to search for player beginning with a string using like
def search_player(player_name):
    try:
        cursor.execute("SELECT * FROM Player WHERE User_Name LIKE %s", (player_name + "%"))
        result = cursor.fetchall()
        if result is None:
            print("Player not found")
            return
        print(result)
    except pymysql.Error as err:
        print(f"Error: {err}")
    return
# similar function for clan name
def search_clan(clan_name):
    try:
        cursor.execute("SELECT * FROM Clan WHERE Clan_Name LIKE %s", (clan_name + "%"))
        result = cursor.fetchall()
        if result is None:
            print("Clan not found")
            return
        print(result)
    except pymysql.Error as err:
        print(f"Error: {err}")
    return

In [ ]:
def success_rate(player_id):
    try:
        # Get number of battles won by player and number of battles played by player
        won = 0
        played = 0

        # Join Battle and Player tables
        query = """
            SELECT Battle.*, Player.User_Name
            FROM Battle
            INNER JOIN Player ON Battle.PlayerID = Player.PlayerID
            WHERE Battle.WinnerID = %s OR Battle.PlayerID = %s
        """

        cursor.execute(query, (player_id, player_id))
        result = cursor.fetchall()

        if not result:
            print("Player not found or has not participated in any battles.")
            return

        for row in result:
            played += 1
            if row["WinnerID"] == player_id:
                won += 1

        success_rate = (won / played) if played > 0 else 0
        print(f"Player ID: {player_id}, User Name: {result[0]['User_Name']}, Success Rate: {success_rate:.2%}")

    except pymysql.Error as err:
        print(f"Error: {err}")

    return


In [ ]:
success_rate(2)